In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import pandas as pd
from DarkNews import const
from DarkNews import plot_tools as pt
from DarkNews import Cfourvec as Cfv

from alp import plot_tools as pt
from alp import beams
from alp import models
from alp import exp
from alp.exp_dicts import EXPERIMENTS

# Saving Pythia8 events to parquet files for easier use

In [5]:
alp = models.ALP(0.1, 1e4, c_lepton=np.diag([1, 1, 1]))

NUMI_files = ['pythia8_events/soft_120_GeV',
              'pythia8_events/soft_120_GeV_3e3',
              'pythia8_events/soft_120_GeV_2e4']

SPS_files  = ['pythia8_events/soft_400_GeV',
               'pythia8_events/soft_400_GeV_1e4',
               'pythia8_events/soft_400_GeV_2e4']

LHC_files  = ['pythia8_events/soft_13.6_TeV',
               'pythia8_events/soft_13.6_TeV_1e4']

NUMI      = exp.Experiment(NUMI_files, exp_dic=EXPERIMENTS['NoVA'], alp=alp, savemem=False)
SPS        = exp.Experiment(SPS_files, exp_dic=EXPERIMENTS['SHiP'], alp=alp, savemem=False)
LHC  = exp.Experiment(LHC_files, exp_dic=EXPERIMENTS['FASER'], alp=alp, savemem=False)

# Concatenate all tau Pythia files into a single DataFrame and save to file

In [6]:
df_numi = exp.load_events(NUMI_files)
df_numi.to_parquet('pythia8_events/numi_120GeV.parquet', index=False)
df_sps = exp.load_events(SPS_files)
df_sps.to_parquet('pythia8_events/sps_400GeV.parquet', index=False)
df_lhc = exp.load_events(LHC_files)
df_lhc.to_parquet('pythia8_events/lhc_13.6TeV.parquet', index=False)

In [7]:
print("Total number of Pythia8 events:", NUMI.nevents, SPS.nevents, LHC.nevents)

Total number of Pythia8 events: 2226589 6085287 2121099


# Generating events with simplified parameterization of the D(s) pT spectrum

In [ ]:
n_cc = 4.4
delta_n_cc = 1.6*2  

b = 1.41
delta_b = 0.4*2

n_events = '1e6'

df_120 = beams.generate_taus(n_events=float(n_events), p_beam=120, n_exp=n_cc+delta_n_cc, a=0, b=b, as_dataframe=True)
df_120.to_parquet(f"tau_events/df_120GeV_{n_events}_ncentral_bcentral.parquet")

df_120 = beams.generate_taus(n_events=float(n_events), p_beam=120, n_exp=n_cc-delta_n_cc, a=0, b=b-delta_b, as_dataframe=True)
df_120.to_parquet(f"tau_events/df_120GeV_{n_events}_nlow_blow.parquet")

df_120 = beams.generate_taus(n_events=float(n_events), p_beam=120, n_exp=n_cc+delta_n_cc, a=0, b=b+delta_b, as_dataframe=True)
df_120.to_parquet(f"tau_events/df_120GeV_{n_events}_nhigh_bhigh.parquet")

df_120 = beams.generate_taus(n_events=float(n_events), p_beam=120, n_exp=n_cc-delta_n_cc, a=0, b=b+delta_b, as_dataframe=True)
df_120.to_parquet(f"tau_events/df_120GeV_{n_events}_nlow_bhigh.parquet")

df_120 = beams.generate_taus(n_events=float(n_events), p_beam=120, n_exp=n_cc+delta_n_cc, a=0, b=b-delta_b, as_dataframe=True)
df_120.to_parquet(f"tau_events/df_120GeV_{n_events}_nhigh_blow.parquet")


In [ ]:
n_cc = 5.81
delta_n_cc = 0.28*2

b = 0.96
delta_b = 0.06*2

n_events = '1e6'

df_400 = beams.generate_taus(n_events=float(n_events), p_beam=400, n_exp=n_cc, a=0, b=b, as_dataframe=True)
df_400.to_parquet(f"tau_events/df_400GeV_{n_events}_ncentral_bcentral.parquet")

df_400 = beams.generate_taus(n_events=float(n_events), p_beam=400, n_exp=n_cc-delta_n_cc, a=0, b=b-delta_b, as_dataframe=True)
df_400.to_parquet(f"tau_events/df_400GeV_{n_events}_nlow_blow.parquet")

df_400 = beams.generate_taus(n_events=float(n_events), p_beam=400, n_exp=n_cc+delta_n_cc, a=0, b=b+delta_b, as_dataframe=True)
df_400.to_parquet(f"tau_events/df_400GeV_{n_events}_nhigh_bhigh.parquet")

df_400 = beams.generate_taus(n_events=float(n_events), p_beam=400, n_exp=n_cc-delta_n_cc, a=0, b=b+delta_b, as_dataframe=True)
df_400.to_parquet(f"tau_events/df_400GeV_{n_events}_nlow_bhigh.parquet")

df_400 = beams.generate_taus(n_events=float(n_events), p_beam=400, n_exp=n_cc+delta_n_cc, a=0, b=b-delta_b, as_dataframe=True)
df_400.to_parquet(f"tau_events/df_400GeV_{n_events}_nhigh_blow.parquet")

In [13]:
df = pd.read_parquet("tau_events/df_120GeV_1e6_ncentral_bcentral.parquet")
df = pd.read_parquet("tau_events/df_400GeV_1e6_ncentral_bcentral.parquet")